**Let's Figure Out how we can embed the text and floating point parameters**

Embedding is the method in which a continue value or a discrete variable can be represented in a continuous vector. Embedding is a highly utilized method in machine translation and entity embedding for categorical variables. 

An embedding is a mapping of a discrete — categorical — variable to a vector of continuous numbers. In the context of neural networks, embeddings are low-dimensional, learned continuous vector representations of discrete variables.

The primary reasons for embedding content is to the nearest neighbors within a given embedding space, as an input for a supervised task, or visualization of categories. 

In a nutshell, NN embeddings can easily take all 37,000 book articles on Wikipedia and represent each one using only 50 numbers in a vector.

**MLP AutoEncoder**

An autoencoder is not used for supervised learning. We will no longer try to predict something about our input. Instead, an autoencoder is considered a generative model: it learns a distributed representation of our training data, and can even be used to generate new instances of the training data.

An autoencoder model contains two components:

An encoder that takes an image as input, and outputs a high-dimensional embedding (representation) of the flaot.
A decoder that takes the high-dimensional embedding, and reconstructs the float.

In [1]:
import torch 
import torch.nn as nn
from torch.nn import Embedding
import os.path

pathAE = "/Users/omoruyiatekha/Documents/GitHub/Referential Language for CAD/autoencoder.pth"

class MLPAutoE(nn.Module):
  # Initialize Neural Network
  def __init__(self):
    super().__init__()

    # Encoder 
    self.encoder = nn.Sequential(
      nn.Linear(1, 5),
      nn.ReLU(),
      nn.Linear(5, 10),
      
    )

    # Decoder
    self.decoder = nn.Sequential(
      nn.Linear(10, 5),
      nn.ReLU(),
      nn.Linear(5, 1),
      
    )
  
  def forward(self, x):
    encoded = self.encoder(x) 
    decoded = self.decoder(encoded)

    return decoded

    # Note the last layer is range [0, infinity], so we will apply a relu function

# Load the Model
autoencoder = MLPAutoE()
model = autoencoder 
model.load_state_dict(torch.load(pathAE))


<All keys matched successfully>

**Creating the Token Embedder**

Then what I had in mind was:
<CLS> ==> [tokenizer embedding module] ==> embedding in R^d


In [2]:
import torch 
import torch.nn as nn
from torch.nn import Embedding
pathE = "/Users/omoruyiatekha/Documents/GitHub/Referential Language for CAD/embedder.pth"


""" The Embedding takes in the tokenized word and returns a R^d vector where d = 10. Our Language has 10 different words. within the dictionary
    Although we are only using 7 different words in our language to begin with. 
    
    Instead of generating random hidden layers, we can use the given weights to generate constant embeddings for our various words. This ensures, 
    that the embeddings are consistent and do not change. 

    The Code above is the same as the code below. The only difference is that the weights are given to the embedding layer. Furthermore,
    the number_embeddings = 10, d is the length of the emvedding vector. d = 10.
"""

" Create Embedding and set weights as a constant. "
weight = torch.FloatTensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 2.0938,  0.6383,  0.4265, -0.1057, -0.9903,  0.0483,  2.0725,  2.7587,
          1.9364,  0.5201],
        [-0.7956, -0.1561, -0.6276,  2.1033, -2.6051,  0.1143,  1.1203, -0.5265,
          0.2124,  0.3033],
        [ 0.2314, -1.1640, -0.5984,  0.5727, -1.4985,  0.0431, -0.2939,  1.4067,
          0.9089, -0.5380],
        [ 0.2106,  0.5994,  2.9547,  1.3907, -0.8808, -0.3567,  0.0752, -0.1998,
         -1.2545, -1.0933],
        [-0.6807,  0.1290, -0.8570, -0.0191,  1.2586, -0.6069,  0.5996,  0.7852,
         -0.1459, -0.0803],
        [ 1.0895, -0.9596,  0.0159,  0.3027,  1.2430,  0.7185,  0.1470,  0.2592,
          1.7928, -1.2126],
        [ 1.2758, -0.3504, -0.2135, -1.0314,  0.1238, -2.0741,  1.2185, -0.3169,
         -0.4115,  0.5401],
        [-1.7623, -0.3535,  0.2283, -0.3083, -1.4517, -1.0330, -0.4992, -1.3504,
          0.0828, -0.4922],
        [-0.7633, -0.5626, -0.1650,  3.0397, -1.5256, -0.8837,  1.1996, -1.1857,
         -1.1322, -0.6586]])


embedding = nn.Embedding.from_pretrained(weight)
input = torch.LongTensor([7])
embedding(input)

torch.save(embedding.state_dict(), pathE)

**Creating a Tokenizer and LxD Matrix for Transformers**

Tokenization — this preprocessing step means transforming unstructured Natural Language input in something better structured (in computer terms). The main idea is to break the textual input into fragments that contain granular, yet useful data — these are called Tokens. 

Following the tokenization process and the MLP encoding into (1,10) vectors. We have to create a LxD matrix to pass into the nn.transformer.


In [3]:
import torch 
import torch.nn as nn
from torch.nn import Embedding
import numpy as np


# Load the Models
embeder = nn.Embedding(10, 10)
embeder.load_state_dict(torch.load(pathE))
#embeder(input)

# Load the Model
mlp = autoencoder = MLPAutoE()
mlp.load_state_dict(torch.load(pathAE))
#mlp.encoder(t)

dictionary = """[CLS] Cylinder Sphere Cube [SEP] { } ,"""
sentence = "[CLS] Cube { 23.2,13,14.2 }"
tokens = dictionary.split()
d = 10


def processText(text):
  text = text.replace('{', '{ ')
  text = text.replace('}', '} ')
  updatedText = text.replace(',', ' ').split(' ')
  updatedText = ' '.join(updatedText).split()
  return updatedText

class token:
  def __init__(self, tokens):
    self.tokens = tokens.split()
    self.embedding = nn.Embedding(50, d) 
    
  def encode(self, sentence):

    split = processText(sentence)
    encoded = []
    
    for word in split:
      if word in self.tokens:
        encoded.append(self.tokens.index(word))
      else:
        encoded.append(word)

    return encoded

  def decode(self, encoded):
    decoded = []
    print(encoded)
    for i in encoded:
      
      if type(i) == str:
        decoded.append(float(i))
      else:
        decoded.append(self.tokens[i])
    return decoded


  def tensorEncoded(self, encoded):
    newList = []
    tensorList = []
    for i in encoded:
      newList.append(float(i))

    for i in newList:
      tensorList.append(torch.tensor([i]))

    return tensorList

  def tensorList(self, encoded):
    array = []
    for i in encoded:
      array.append(np.array(i)[0])

    return(array)

  def encodeD(self, sentence):
    encoded = self.encode(sentence)
    tensor = self.tensorEncoded(encoded)
    D = torch.tensor([]) 

    for i in range(len(tensor)):
      if (i < 3) or (i == len(tensor) - 1):
        temp = embeder(tensor[i].to(torch.int32))
        #print(temp)
        D = torch.cat((D, temp), 0)
        #temp = embeder(tensor[i])
        #temp = tensor[i].view(1, 10) 
      else:
        temp = (mlp.encoder(tensor[i])).view(1, 10)
        #print(temp)
        D = torch.cat((D, temp), 0)
        #print(tensor[i])
        #tensor[i] = tensor[i]


    return D


tokens = token(dictionary)
encoded = tokens.encode(sentence)
decoded = tokens.decode(encoded)
print(decoded)
TensorEncoded = tokens.tensorEncoded(encoded)

D = tokens.encodeD(sentence)
print(D)

#print(D)
#print(tokens.tensorList(encoded))

[0, 3, 5, '23.2', '13', '14.2', 6]
['[CLS]', 'Cube', '{', 23.2, 13.0, 14.2, '}']
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 2.3140e-01, -1.1640e+00, -5.9840e-01,  5.7270e-01, -1.4985e+00,
          4.3100e-02, -2.9390e-01,  1.4067e+00,  9.0890e-01, -5.3800e-01],
        [-6.8070e-01,  1.2900e-01, -8.5700e-01, -1.9100e-02,  1.2586e+00,
         -6.0690e-01,  5.9960e-01,  7.8520e-01, -1.4590e-01, -8.0300e-02],
        [ 7.0714e+00,  5.7949e+00,  4.5472e+00, -8.8286e+00, -7.1117e+00,
          9.0574e+00,  1.7471e+01,  3.9549e+00, -6.7391e+00, -3.8288e+00],
        [ 4.0820e+00,  3.3468e+00,  2.4925e+00, -4.5453e+00, -3.5795e+00,
          4.9992e+00,  9.6312e+00,  1.8883e+00, -3.7138e+00, -2.2015e+00],
        [ 4.4337e+00,  3.6348e+00,  2.7342e+00, -5.0492e+00, -3.9951e+00,
          5.4767e+00,  1.0554e+01,  2.1314e+00, -4.0697e+00, -2.3929e+00],
        [ 1.0895e+00, -9.

In [17]:
encoder_layer = nn.TransformerEncoderLayer(d_model=10, nhead=10)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
out = transformer_encoder(D)
#print(src)
#print(out)
print(out)

tensor([[ 2.0710e+00,  3.0823e-01, -1.0232e+00,  6.8680e-01, -1.0312e-01,
         -1.0767e+00,  2.9498e-01,  8.7588e-01, -9.6946e-01, -1.0644e+00],
        [ 2.0922e+00, -1.2261e-01, -9.4206e-01,  4.6095e-01, -1.1485e+00,
         -3.6325e-01, -6.3598e-04,  1.3323e+00, -1.9270e-01, -1.1156e+00],
        [ 1.6321e+00,  5.3252e-01, -8.8919e-01,  7.3114e-01, -5.3378e-01,
         -9.1903e-01,  6.3735e-01,  1.1283e+00, -8.2143e-01, -1.4980e+00],
        [ 2.0810e+00,  1.9103e-01, -1.3267e-01,  6.1315e-01, -9.9281e-01,
         -3.9540e-01,  3.5744e-01,  7.8350e-01, -9.2270e-01, -1.5825e+00],
        [ 2.0336e+00, -2.8687e-02, -1.8967e-01,  5.5018e-01, -1.1590e+00,
         -2.7159e-01,  2.9892e-01,  1.0606e+00, -7.8574e-01, -1.5086e+00],
        [ 1.9025e+00,  3.1869e-01, -1.4029e-01,  6.7750e-01, -3.0493e-01,
         -6.4414e-01,  6.5119e-02,  9.4295e-01, -9.4489e-01, -1.8725e+00],
        [ 2.0901e+00,  2.6380e-01, -7.0914e-01,  5.5183e-01, -9.4635e-01,
         -2.1608e-01, -2.8275e-0

**Training the Embedding Models**

Below is the code that allows us to embed our sentence and floating parameters to a higer dimensional space

In [5]:
# Let's Train the AutoEncoder on a small generated set of data that should not be too terrible.
def train(model, num_epochs):

  criterion = nn.MSELoss()
  autoencoder = model
  optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3, weight_decay=1e-3)
  outputs = []
  data_set = torch.tensor([[1.65], [2], [2.3], [54], [1.3], [43.2], [120.324], [32.123]])

  for epoch in range(num_epochs):
    for p_i in data_set:
      p_pred = autoencoder(p_i)
      loss = criterion(p_i, p_pred)

      
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      #print("p_i: ", p_i, " predicted: ", p_pred)
      if epoch % 10 == 0:
        print(f'Epoch:{epoch+1}, Loss:{loss.item():4f}')

    outputs.append((epoch, p_i, p_pred))
  
  return outputs

train(autoencoder, 500)
torch.save(autoencoder.state_dict(), path)


Epoch:1, Loss:0.000001
Epoch:1, Loss:0.001319
Epoch:1, Loss:0.000083
Epoch:1, Loss:0.118781
Epoch:1, Loss:0.000000
Epoch:1, Loss:0.145028
Epoch:1, Loss:0.735905
Epoch:1, Loss:0.003593
Epoch:11, Loss:0.000260
Epoch:11, Loss:0.000247
Epoch:11, Loss:0.000232
Epoch:11, Loss:0.005157
Epoch:11, Loss:0.000297
Epoch:11, Loss:0.002096
Epoch:11, Loss:0.001132
Epoch:11, Loss:0.000203
Epoch:21, Loss:0.000553
Epoch:21, Loss:0.000088
Epoch:21, Loss:0.000050
Epoch:21, Loss:0.759409
Epoch:21, Loss:0.000052
Epoch:21, Loss:0.876105
Epoch:21, Loss:2.259578
Epoch:21, Loss:0.357284
Epoch:31, Loss:0.000229
Epoch:31, Loss:0.000202
Epoch:31, Loss:0.000176
Epoch:31, Loss:0.000426
Epoch:31, Loss:0.000159
Epoch:31, Loss:0.000199
Epoch:31, Loss:0.000179
Epoch:31, Loss:0.000021
Epoch:41, Loss:0.000000
Epoch:41, Loss:0.000001
Epoch:41, Loss:0.000004
Epoch:41, Loss:0.001920
Epoch:41, Loss:0.000003
Epoch:41, Loss:0.001667
Epoch:41, Loss:0.003011
Epoch:41, Loss:0.000443
Epoch:51, Loss:0.000879
Epoch:51, Loss:0.000055


NameError: name 'path' is not defined

In [ ]:
# Check the model
t = torch.tensor([[1], [2], [2.3], [54], [1.3], [43.2], [120.324], [32.123]])
P_t = autoencoder.encoder(t)
autoencoder.decoder(P_t)

# Load the Model
model = autoencoder = MLPAutoE()
model.load_state_dict(torch.load(path))
model.encoder(t)

tensor([[ 5.6518e-01,  4.6659e-01,  7.5284e-02,  4.9395e-01,  5.7597e-01,
          2.2485e-01,  4.0789e-01, -5.4314e-01, -1.5475e-01, -2.8697e-01],
        [ 8.5826e-01,  7.0661e-01,  2.7672e-01,  7.4017e-02,  2.2968e-01,
          6.2271e-01,  1.1765e+00, -3.4052e-01, -4.5134e-01, -4.4652e-01],
        [ 9.4618e-01,  7.7861e-01,  3.3715e-01, -5.1963e-02,  1.2579e-01,
          7.4207e-01,  1.4071e+00, -2.7974e-01, -5.4032e-01, -4.9438e-01],
        [ 1.6098e+01,  1.3187e+01,  1.0751e+01, -2.1763e+01, -1.7778e+01,
          2.1312e+01,  4.1144e+01,  1.0196e+01, -1.5874e+01, -8.7428e+00],
        [ 6.5311e-01,  5.3860e-01,  1.3572e-01,  3.6797e-01,  4.7208e-01,
          3.4421e-01,  6.3847e-01, -4.8235e-01, -2.4373e-01, -3.3484e-01],
        [ 1.2933e+01,  1.0595e+01,  8.5759e+00, -1.7227e+01, -1.4038e+01,
          1.7015e+01,  3.2843e+01,  8.0072e+00, -1.2671e+01, -7.0197e+00],
        [ 3.5532e+01,  2.9104e+01,  2.4109e+01, -4.9610e+01, -4.0740e+01,
          4.7698e+01,  9.2118e+0

**MLP Encoder**

Now if you want to share the same MLP encoder for all numerical inputs (regardless of whether they are radius or height measurements,  etc), then your MLP encoder would take in R^1 and output R^d. These are floating point values, for both input and output.

tensor([1.])
tensor([2.])
tensor([2.3000])
tensor([54.])
tensor([1.3000])
tensor([43.2000])
tensor([120.3240])
tensor([32.1230])


In [ ]:
import torch 
import torch.nn as nn
from torch.nn import Embedding

# MLP to take in a 1D float and encode it into a higher dimensional space.
# Input:  Tensor of R^1  (Shape: (1, ))
# Output: Tensor of R^10 (Shape: (1, 10))

# Create Encoder and Decoder Model with some random hidden weights and layers

inputsize = (1,)
outputsize = (1,10)
t = torch.tensor([[1.5]])

class MLPEncoder(nn.Module):
  # Initialize Neural Network
  def __init__(self):
    super(MLPEncoder, self).__init__()
    self.l1 = nn.Linear(1, 5)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(5, 10)

  def forward(self, x):
    output = self.l1(x) 
    output = self.relu(output)
    output = self.l2(output)
    return output

net = MLPEncoder()

outputs = net(t)
print(outputs)
outputs = net(t)
print(outputs)

  

tensor([[-0.3786,  0.3023,  0.3095,  0.0118,  0.4308,  0.1019,  0.0553,  0.1434,
          0.1401, -0.3750]], grad_fn=<AddmmBackward0>)
tensor([[-0.3786,  0.3023,  0.3095,  0.0118,  0.4308,  0.1019,  0.0553,  0.1434,
          0.1401, -0.3750]], grad_fn=<AddmmBackward0>)
